In [ ]:
#|default_exp peak_pattern_puzzle_solver 

# Solving the peak pattern puzzle 

> Which elements are hidden in our data cube?

In [ ]:
#|hide 
%load_ext autoreload
%autoreload 2 

In [ ]:
#|hide 
import os 

In [ ]:
#|hide 
os.chdir('/home/frank/Work/Projecten/DoRe/viz/raw_nmf')

In [ ]:
#|hide 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib notebook

In the previous section we saw how to explain the presence of specific in the individual hotmax spectra by  comparison with the instrument and element peak patterns by hand. Tedious work, but not too difficult. Let's now try to extract the algorithm for solving the peak pattern puzzle. As an example let's walk through hotmax spectrum **#4**. In the plot we see thirteen peaks that exceed the noise threshold. We need to explain these peaks away, one by one... 

In [ ]:
from maxrf4u import plot_puzzle, HotmaxAtlas, DataStack 

In [ ]:
fig, ax, ax1 = plot_puzzle('RP-T-1898-A-3689.datastack', 4, color_select=[])
ax.set_title('Peak pattern atlas');

..

Initially, in hotmax spectrum **#4** there are thirteen detected peaks that we need to explain. These peaks are numbered from highest to lowest. Let's see how we can explain them away one by one. To read the thirteen peak indexes, we can use the `DataStack.read_list()` method. Note that in this case we can not use the standard `DataStack.read()` method because the data in the datastack is stored as a ragged list.  

Here are the 13 sub peak indexes of hotmax spectrum **#4**.

In [ ]:
ds = DataStack('RP-T-1898-A-3689.datastack')

In [ ]:
subpeak_idxs = ds.read_list('hotmax_subpeak_idxs_list')[4]
subpeak_idxs

[95, 1981, 466, 735, 427, 329, 2108, 800, 2206, 1360, 152, 1522, 933]

Let's convert these channel indexes into energies first. 

In [ ]:
x_keVs = ds.read('maxrf_energies')

In [ ]:
subpeak_keVs = x_keVs[subpeak_idxs]
list(enumerate(subpeak_keVs))

[(0, -0.028648044952708673),
 (1, 18.895639680305266),
 (2, 3.693998161871633),
 (3, 6.393167513989552),
 (4, 3.3026687762485896),
 (5, 2.3193282687855556),
 (6, 20.16996870528287),
 (7, 7.0453831566946254),
 (8, 21.153309212745903),
 (9, 12.664471770769104),
 (10, 0.5432949032655865),
 (11, 14.289993834126363),
 (12, 8.379916702537315)]

Now, we need to consult the peak pattern atlas with all element starting with the instrument peak pattern. 

In [ ]:
from maxrf4u import get_patterns, get_instrument_pattern
import numpy as np

In [ ]:
instr_ptrn = get_instrument_pattern('RP-T-1898-A-3689.datastack')
instr_ptrn

{'name': 'INSTRUMENT',
 'instrument_peaks': array([-0.02864804, 18.82674463, 20.20010005, 20.99840233, 22.72136068])}

We now need to check which peaks in the hotmax spectrum match the instrument peaks. 

In [ ]:
instr_keVs = instr_ptrn['instrument_peaks']
instr_keVs

array([-0.02864804, 18.82674463, 20.20010005, 20.99840233, 22.72136068])

Let's see which peaks match within a distance of 0.1 keV. 

In [ ]:
delta_keV = 0.1
distance_matrix = np.sqrt((subpeak_keVs[:, None] - instr_keVs[None, :])**2)
is_nearby = distance_matrix < delta_keV
peak_matches = np.argwhere(is_nearby)
peak_matches

array([[0, 0],
       [1, 1],
       [6, 2]])

Let's color the markers on these matched instrument peaks red...

In [ ]:
y_hot = ds.read('hotmax_spectra')[4] 

match_idxs = peak_matches[:, 0]
match_x = subpeak_keVs[match_idxs]
match_y = y_hot[np.array(subpeak_idxs)[match_idxs]]

In [ ]:
subpeak_idxs[]

[95, 1981, 466, 735, 427, 329, 2108, 800, 2206, 1360, 152, 1522, 933]

In [ ]:
from maxrf4u import HotmaxAtlas

In [ ]:
hma = HotmaxAtlas('RP-T-1898-A-3689.datastack')

In [ ]:
ax, ann_list = hma.plot_spectrum(4)

ax.scatter(match_x, match_y, facecolor='r', edgecolor='w', marker='X', s=100)
ax.set_title('Matching instrument peaks');

..